In [ ]:
import numpy as np
from copy import deepcopy

class GameState:
    def __init__(self):
        self._state = np.zeros((3,2))
        self._state[(2,1)] = np.nan
        
        self._turn = 0
        self._last_move = [None, None]
    
    def forecast_move(self, move):
        """ Return a new board object with the specified move
        applied to the current game state.
        
        Parameters
        ----------
        move: tuple
            The target position for the active player's next move
        """
        assert move in self.get_legal_moves(), 'Illegal move'
        
        new_board = deepcopy(self)
        new_board._state[move] = 1.
        new_board._last_move[self._turn] = move
        new_board._turn ^= 1
        
        return new_board
    
    def get_legal_moves(self):
        """ Return a list of all legal moves available to the
        active player.  Each player should get a list of all
        empty spaces on the board on their first move, and
        otherwise they should get a list of all open spaces
        in a straight line along any row, column or diagonal
        from their current position. (Players CANNOT move
        through obstacles or blocked squares.) Moves should
        be a pair of integers in (column, row) order specifying
        the zero-indexed coordinates on the board.
        """
        if np.nansum(self._state) < 2:
            return self._get()
        
        moves = self._get()
        legal_moves = []

        rays = [(1, 0), (1, -1), (0, -1), (-1, -1),
                (-1, 0), (-1, 1), (0, 1), (1, 1)]
        
        for x, y in rays:
            loc = self._last_move[self._turn]
            while True:
                loc = loc[0]+x, loc[1]+y
                if loc in moves:
                    legal_moves.append(loc)
                else:
                    break
    
        return legal_moves

    def _get(self, value=0.):
        return [(x,y) for x in range(3) for y in range(2) if self._state[x,y] == value]
    
    def __repr__(self):
        return str(self._state.T)

g = GameState()
print(g.get_legal_moves())

g1 = g.forecast_move((2,0))
print(g1.get_legal_moves())

g2 = g1.forecast_move((1,0))
print(g2.get_legal_moves())
print(g2)

[(1,1)] == g2.get_legal_moves()

g1 = g2.forecast_move((1,1))
print(g1.get_legal_moves())
print(g1)

In [ ]:
print("Creating empty game board...")
g = GameState()

print("Getting legal moves for player 1...")
p1_empty_moves = g.get_legal_moves()
print("Found {} legal moves.".format(len(p1_empty_moves or [])))

print("Applying move (0, 0) for player 1...")
g1 = g.forecast_move((0, 0))

print("Getting legal moves for player 2...")
p2_empty_moves = g1.get_legal_moves()
if (0, 0) in set(p2_empty_moves):
    print("Failed\n  Uh oh! (0, 0) was not blocked properly when " +
          "player 1 moved there.")
else:
    print("Everything looks good!")

* Assumption 1: a state is terminal if the active player has no remaining moves
* Assumption 2: the board utility is -1 if it terminates at a max level, and +1 if it terminates at a min level

In [1]:
def terminal_test(gameState):
    """ Return True if the game is over for the active player
    and False otherwise.
    """
    return not bool(gameState.get_legal_moves())  # by Assumption 1
    #if gameState.get_legal_moves():
    #    return False
    #return True

def min_value(gameState):
    """ Return the value for a win (+1) if the game is over,
    otherwise return the minimum value over all legal child
    nodes.
    """
    if terminal_test(gameState):
        return 1
    
    v = np.inf
    for m in gameState.get_legal_moves():
        v = min(v, max_value(gameState.forecast_move(m)))
    return v

def max_value(gameState):
    """ Return the value for a loss (-1) if the game is over,
    otherwise return the maximum value over all legal child
    nodes.
    """
    if terminal_test(gameState):
        return -1

    v = -np.inf
    for m in gameState.get_legal_moves():
        v = max(v, min_value(gameState.forecast_move(m)))
    return v

g = GameState()

print("Calling min_value on an empty board...")
v = min_value(g)

if v == -1:
    print("min_value() returned the expected score!")
else:
    print("Uh oh! min_value() did not return the expected score.")

NameError: name 'GameState' is not defined

In [ ]:
def minimax_decision(gameState):
    """ Return the move along a branch of the game tree that
    has the best possible value.  A move is a pair of coordinates
    in (column, row) order corresponding to a legal move for
    the searching player.
    
    You can ignore the special case of calling this function
    from a terminal state.
    """
    criteria = lambda m : min_value(gameState.forecast_move(m))
    return max(gameState.get_legal_moves(), key=criteria)

best_moves = set([(0, 0), (2, 0), (0, 1)])
rootNode = GameState()
minimax_move = minimax_decision(rootNode)

print("Best move choices: {}".format(list(best_moves)))
print("Your code chose: {}".format(minimax_move))

if minimax_move in best_moves:
    print("That's one of the best move choices. Looks like your minimax-decision function worked!")
else:
    print("Uh oh...looks like there may be a problem.")